In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB


In [2]:
yelp_raw = pd.read_csv('data/yelp_labelled.txt', 
                       delimiter= '\t', 
                       header=None)


imdb_raw = pd.read_csv('data/imdb_labelled.txt', 
                       delimiter= '\t', 
                       header=None)


amazon_raw = pd.read_csv('data/amazon_cells_labelled.txt', 
                         delimiter= '\t', 
                         header=None
                        )


In [3]:
# set dataframe to source
df = yelp_raw.copy()

# df = imdb_raw.copy()

# df = amazon_raw.copy()


df.columns = ['message', 'negative']

In [4]:
df['negative'] = (df['negative'] == 0)
# Note that if you run this cell a second time everything will become false.
# So... Don't.

In [5]:
# replace punctuation 
df['message'] = df.message.str.replace(r'[^a-zA-Z\d\s:]', '')
# make lower case
df['message'] = df['message'].str.lower()

In [6]:
# split negative messages and combine into one list
negative_words = df.message[df.negative].str.cat(sep=' ').split()

positive_words = df.message[df.negative == False].str.cat(sep=' ').split()

In [7]:
# Unique Words
print('negative:', len(np.unique(negative_words)), ' positive:', len(np.unique(positive_words)))

negative: 1397  positive: 1246


In [8]:
# Create positive words
diff_pos = np.setdiff1d(ar1=positive_words, 
                      ar2=negative_words
                     )

# Create negative words
diff_neg = np.setdiff1d(ar1=positive_words, 
                      ar2=negative_words
                     )

# combine
diff = np.append(diff_pos, diff_neg)

#split
diff = np.random.choice(diff, size=int(len(diff) / 2))

diff = diff_neg

# diff = diff_pos

In [9]:
new = df.copy()

for key in diff:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    new[key] = df['message'].apply(lambda x: key in x)

In [10]:
new = new.iloc[:, 2:]

In [11]:
data = new.copy()
target = df['negative']

In [12]:
data = data.applymap(lambda x: 1 if x == True else x)
data = data.applymap(lambda x: 0 if x == False else x)

In [13]:
words_50 = data.describe().T.sort_values(by='mean', 
                                         ascending=False).head(1000).index.tolist()

In [14]:
new = df.copy()

for key in words_50:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    new[key] = new.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

In [15]:
data = new.iloc[:, 2:]
target = df['negative']

In [16]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(data, target)

# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

# Display our results.
print("Number of mislabeled points out of a total {} points : {}\nAccuracy: {}%".format(
    data.shape[0],
    (target != y_pred).sum(), round((1 - (target != y_pred).sum()/data.shape[0]) * 100, 2)
))

display(pd.crosstab(y_pred, target))

Number of mislabeled points out of a total 1000 points : 319
Accuracy: 68.1%


negative,False,True
row_0,,
False,181,0
True,319,500


In [17]:
cross_val_score(bnb, data, target, cv=5)

array([0.585, 0.585, 0.58 , 0.595, 0.61 ])

# This iteration is exceptional at finding positive reviews because there are many more unique negative words